In [ ]:
!pip install -q pymongo sentence-transformers pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 23.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo.operations import SearchIndexModel
from google.colab import userdata

In [ ]:
# preparing data
data = pd.read_parquet(path='/content/llm-emoji-dataset.parquet')
data.drop(columns='unicode', inplace=True)
data.rename(columns={'short description': 'short_description', 'LLM description': 'long_description'}, inplace=True)
data['short_description'] = data['short_description'].apply(lambda x: x.lower())
data['long_description'] = data['long_description'].apply(lambda x: x.lower())
data.sample(3)

,character,short_description,tags,long_description
571,🤙🏼,call me hand medium-light skin tone,"[call, invitation, contact, communication, han...",this emoji represents a hand with palm facing ...
2278,🧎🏽‍♂‍➡️,man kneeling facing right medium skin tone,"[man, kneeling, right, medium skin tone, humil...",this emoji shows a medium skin tone man kneeli...
2325,👨🏿‍🔧,man mechanic dark skin tone,"[man, mechanic, dark skin tone, profession, re...",this emoji represents a man with a dark skin t...


In [ ]:
# generating embeddings
model = SentenceTransformer('nomic-ai/nomic-embed-text-v1', trust_remote_code=True)
embeddings, characters, short_descriptions, long_descriptions, tags = [], [], [], [], []

for _, i in tqdm(iterable=data.iterrows(), total=data.shape[0]):
  embeddings.append(model.encode(' '.join(i['tags'].tolist()) + ' ' + i['short_description'] + i['long_description']))
  characters.append(i['character'])
  short_descriptions.append(i['short_description'])
  long_descriptions.append(i['long_description'])
  tags.append(i['tags'])

embeddings_df = pd.DataFrame({
    'emoji': characters,
    'tags': tags,
    'short_description': short_descriptions,
    'long_description': long_descriptions,
    'embeddings': embeddings
})
embeddings_df.to_csv('/content/embed_nomic-embed-text-v1.csv')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [ ]:
uri = f"mongodb+srv://a002k:{userdata.get('db_password')}@{userdata.get('app_name')}.5xpbp.mongodb.net/?retryWrites=true&w=majority&appName={userdata.get('app_name')}"
client = MongoClient(uri, server_api=ServerApi('1'))
db = client[userdata.get('db_name')]
coll = db[userdata.get('coll_name')]

In [ ]:
# storing embeddings in mongodb collection
for _, i in tqdm(iterable=embeddings_df.iterrows(), total=embeddings_df.shape[0]):
  coll.insert_one({
      'emoji': i['emoji'],
      'tags': i['tags'].tolist(),
      'short_description': i['short_description'],
      'long_description': i['long_description'],
      'embeddings': i['embeddings'].tolist()
  })

100%|██████████| 5034/5034 [20:20<00:00,  4.12it/s]


In [ ]:
# running vector search
query_embedding = model.encode('computers and hardware')

pipeline = [
   {
      "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding.tolist(),
            "path": "embeddings",
            "exact": True,
            "limit": 5
      }
   },
   {
      "$project": {
         "_id": 0,
         "emoji": 1,
         "tags": 1,
         "score": {
            "$meta": "vectorSearchScore"
         }
      }
   }
]

results = list(coll.aggregate(pipeline))
results

[{'emoji': '💻',
  'tags': ['laptop',
   'computer',
   'technology',
   'portable',
   'work',
   'study',
   'entertainment',
   'keyboard',
   'touchpad',
   'screen'],
  'score': 0.5195567607879639},
 {'emoji': '🖥',
  'tags': ['desktop computer',
   'technology',
   'communication',
   'work',
   'computing',
   'device',
   'digital'],
  'score': 0.5194892883300781},
 {'emoji': '🖥️',
  'tags': ['desktop computer',
   'technology',
   'communication',
   'internet',
   'work',
   'information'],
  'score': 0.5099403858184814},
 {'emoji': '⌨',
  'tags': ['keyboard',
   'input',
   'device',
   'technology',
   'typing',
   'computer',
   'communication'],
  'score': 0.5097317695617676},
 {'emoji': '💾',
  'tags': ['floppy disk',
   'data storage',
   'technology',
   'history',
   'computer',
   'gaming'],
  'score': 0.4969928562641144}]